In [2]:
import math
import random
import time
import json
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w  = Decimal("0.4")
        top_w  = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def clone(self):
        return ChristmasTree(center_x=str(self.center_x), center_y=str(self.center_y), angle=str(self.angle))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def has_overlap(tree_list):
    polys = [t.polygon for t in tree_list]
    tree = STRtree(polys)
    for i, poly in enumerate(polys):
        candidates = tree.query(poly)
        for cand_idx in candidates:
            if cand_idx != i:
                if poly.intersects(polys[cand_idx]):
                    if not poly.touches(polys[cand_idx]):
                         if poly.intersection(polys[cand_idx]).area > 1e-9:
                             return True
    return False

def simulated_annealing(trees, max_time=60, initial_temp=2.0, cooling_rate=0.95):
    current_trees = [t.clone() for t in trees]
    current_score = get_tree_list_side_length(current_trees)
    best_trees = [t.clone() for t in current_trees]
    best_score = current_score
    
    start_time = time.time()
    temp = initial_temp
    
    iter_count = 0
    while time.time() - start_time < max_time:
        iter_count += 1
        neighbor_trees = [t.clone() for t in current_trees]
        
        # More aggressive moves
        idx = random.randint(0, len(neighbor_trees) - 1)
        move_type = random.choice(['shift', 'rotate', 'swap', 'big_shift'])
        
        if move_type == 'shift':
            dx = random.uniform(-0.1, 0.1) * temp
            dy = random.uniform(-0.1, 0.1) * temp
            neighbor_trees[idx].center_x += Decimal(dx)
            neighbor_trees[idx].center_y += Decimal(dy)
            neighbor_trees[idx] = ChristmasTree(neighbor_trees[idx].center_x, neighbor_trees[idx].center_y, neighbor_trees[idx].angle)
            
        elif move_type == 'big_shift':
             # Try moving to a completely random position within current bounds
             bounds = unary_union([t.polygon for t in neighbor_trees]).bounds
             width = bounds[2] - bounds[0]
             height = bounds[3] - bounds[1]
             neighbor_trees[idx].center_x = Decimal(random.uniform(bounds[0], bounds[2]))
             neighbor_trees[idx].center_y = Decimal(random.uniform(bounds[1], bounds[3]))
             neighbor_trees[idx] = ChristmasTree(neighbor_trees[idx].center_x, neighbor_trees[idx].center_y, neighbor_trees[idx].angle)

        elif move_type == 'rotate':
            d_theta = random.uniform(-45, 45) * temp # Larger rotation
            neighbor_trees[idx].angle += Decimal(d_theta)
            neighbor_trees[idx] = ChristmasTree(neighbor_trees[idx].center_x, neighbor_trees[idx].center_y, neighbor_trees[idx].angle)
            
        elif move_type == 'swap' and len(neighbor_trees) > 1:
            idx2 = random.randint(0, len(neighbor_trees) - 1)
            if idx != idx2:
                neighbor_trees[idx].center_x, neighbor_trees[idx2].center_x = neighbor_trees[idx2].center_x, neighbor_trees[idx].center_x
                neighbor_trees[idx].center_y, neighbor_trees[idx2].center_y = neighbor_trees[idx2].center_y, neighbor_trees[idx].center_y
                neighbor_trees[idx] = ChristmasTree(neighbor_trees[idx].center_x, neighbor_trees[idx].center_y, neighbor_trees[idx].angle)
                neighbor_trees[idx2] = ChristmasTree(neighbor_trees[idx2].center_x, neighbor_trees[idx2].center_y, neighbor_trees[idx2].angle)

        if has_overlap(neighbor_trees):
            continue
            
        neighbor_score = get_tree_list_side_length(neighbor_trees)
        
        delta = neighbor_score - current_score
        if delta < 0 or random.random() < math.exp(-delta / temp):
            current_trees = neighbor_trees
            current_score = neighbor_score
            
            if current_score < best_score:
                best_score = current_score
                best_trees = [t.clone() for t in current_trees]
                print(f"New best score for N={len(trees)}: {best_score}")
                
        temp *= cooling_rate
        if temp < 0.001:
            temp = initial_temp # Restart
        
    return best_trees, best_score

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[int(group_id)] = tree_list
    return dict_of_tree_list

def write_submission(dict_of_tree_list, out_file):
    rows = []
    sorted_keys = sorted(dict_of_tree_list.keys())
    for group_name in sorted_keys:
        tree_list = dict_of_tree_list[group_name]
        for item_id, tree in enumerate(tree_list):
            rows.append(
                {
                    "id": f"{group_name}_{item_id}",
                    "x": f"s{tree.center_x}",
                    "y": f"s{tree.center_y}",
                    "deg": f"s{tree.angle}",
                }
            )
    pd.DataFrame(rows).to_csv(out_file, index=False)

def main():
    with open("worst_ns.json", "r") as f:
        worst_ns = json.load(f)
    
    target_ns = worst_ns[:5]
    print(f"Targeting Ns: {target_ns}")
    
    current_submission = parse_csv("submission.csv")
    improved_submission = current_submission.copy()
    
    for n in target_ns:
        print(f"Optimizing N={n}...")
        trees = current_submission[n]
        optimized_trees, score = simulated_annealing(trees, max_time=120) # Increased time
        
        old_score = get_tree_list_side_length(trees)
        if score < old_score:
            print(f"IMPROVED N={n}: {old_score} -> {score}")
            improved_submission[n] = optimized_trees
        else:
            print(f"No improvement for N={n}")
            
    write_submission(improved_submission, "submission_sa.csv")
    print("Saved submission_sa.csv")

if __name__ == "__main__":
    main()

Targeting Ns: [1, 2, 3, 5, 4]


Optimizing N=1...


No improvement for N=1
Optimizing N=2...


No improvement for N=2
Optimizing N=3...


No improvement for N=3
Optimizing N=5...


No improvement for N=5
Optimizing N=4...


No improvement for N=4
Saved submission_sa.csv
